Pour se connecter à ce notebook 
Besoin de lancer  l'image singularity de : _Containers/shortcakelight.sif

https://github.com/rnakato/ShortCake/tree/master
Est conçu spécifiquement pour les analyses de données single cell

Pour lancer l'image singularity et pouvoir se connecter aux notebooks jupyter:

singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh

puis faire le lien ssh sur l'ordi avec cette commande  que l'on ecrit dans le terminal local: 
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)

si probleme de connection 

ls -a
rm -r .vscode-server
puis se remet sur vscode 
et refait la liaison avec ssh remote (et reinstaller les extensions...)


In [ ]:
# Telechargement des données preocessées  de l'article de Kuchina et al: 
Microbial single-cell RNA sequencing by split-pool barcoding
https://doi.org/10.1126/science.aba5257

#protocole 2024: 
https://doi.org/10.1038/s41596-024-01007-w

#les données de séquençage brute sont disponible sur SRA : 
The raw sequencing files are available at the Sequence Read Archive:
    - GSM4594094 : https://www.ncbi.nlm.nih.gov/sra/SRX8485151%5baccn%5d
    - GSM4594095 : https://www.ncbi.nlm.nih.gov/sra/SRX8485152%5baccn%5d
    - GSM4594096 : https://www.ncbi.nlm.nih.gov/sra/SRX8485153%5baccn%5d

Processed data were submitted to Gene Expression Omnibus, with accession number GSE151940.

#Les données processées sont disponibles sur le site de GEO:
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE151940


SUPPLEMENTARY MATERIALS 
https://www.science.org/doi/10.1126/science.aba5257

In [21]:
# afficher le repertoire courant
import os
print(os.getcwd())


/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq


In [23]:
%%bash  

# Vérifier si le dossier cible existe, sinon le créer
if [ ! -d "7_Article/data/GSE" ]; then
  echo "Le dossier 7_Article/data/GSE n'existe pas. Création du dossier."
  mkdir -p 7_Article/data/GSE
fi

# URL du fichier à télécharger
URL="https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE151940&format=file"
OUTPUT="7_Article/data/GSE/GSE151940_RAW.tar"

# Vérifier si l'URL est accessible
if curl --head --silent --fail "$URL" > /dev/null; then
  echo "L'URL est accessible, téléchargement en cours..."
  wget  --no-check-certificate -O $OUTPUT $URL

  # Vérifier si le téléchargement a réussi
  if [ -f "$OUTPUT" ]; then
    echo "Téléchargement réussi et fichier sauvegardé sous $OUTPUT"
  else
    echo "Erreur : le téléchargement n'a pas réussi."
    exit 1
  fi
else
  echo "Erreur : l'URL n'est pas accessible."
  exit 1
fi


Le dossier 7_Article/data/GSE n'existe pas. Création du dossier.


L'URL est accessible, téléchargement en cours...


--2025-02-16 12:58:51--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE151940&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
  Self-signed certificate encountered.
HTTP request sent, awaiting response... 200 OK
Length: 7208960 (6.9M) [application/x-tar]
Saving to: ‘7_Article/data/GSE/GSE151940_RAW.tar’

     0K .......... .......... .......... .......... ..........  0%  295K 24s
    50K .......... .......... .......... .......... ..........  1%  582K 18s
   100K .......... .......... .......... .......... ..........  2% 29.1M 12s
   150K .......... .......... .......... .......... ..........  2% 3.24M 9s
   200K .......... .......... .......... .......... ..........  3%  708K 9s
   250K .......... .......... .......... .......... ..........  4% 26.5M 8s
   300K .......... .......... .......... .......... ..........  4% 23.8M 7s
   3

Téléchargement réussi et fichier sauvegardé sous 7_Article/data/GSE/GSE151940_RAW.tar


In [24]:
%%bash
ls -l 7_Article/data/GSE/GSE151940_RAW.tar

# Créer le dossier de décompression s'il n'existe pas
if [ ! -d "7_Article/data/GSE_untar" ]; then
  mkdir -p "7_Article/data/GSE_untar"
fi

# Vérifier si le fichier existe
if [ ! -f "7_Article/data/GSE/GSE151940_RAW.tar" ]; then
  echo "Le fichier .tar n'existe pas."
  exit 1
fi

# Décompresser le fichier
tar -xvf "7_Article/data/GSE/GSE151940_RAW.tar" -C "7_Article/data/GSE_untar"

-rw-r--r-- 1 vgoupille cnrs_umr6553 7208960 Feb 16 12:58 7_Article/data/GSE/GSE151940_RAW.tar
GSM4594094_M11_barcodes.csv.gz


GSM4594094_M11_dcm.csv.gz
GSM4594094_M11_genes.csv.gz
GSM4594095_mRNA_M14_200.csv.gz
GSM4594095_mRNA_M14_ODannotation.csv.gz
GSM4594095_mRNA_M14_gene_names.csv.gz
GSM4594096_mRNA_M15_200.csv.gz
GSM4594096_mRNA_M15_ODannotation.csv.gz
GSM4594096_mRNA_M15_gene_names.csv.gz


In [28]:
%%bash

# répertoire contenant les fichiers .gz
repertoire="7_Article/data/GSE_untar"

# répertoire de destination
repertoire_dest="7_Article/data/GSE_extracted"

# Vérifier si le répertoire de destination existe, sinon le créer
if [ ! -d "$repertoire_dest" ]; then
    mkdir -p "$repertoire_dest"
    echo "Répertoire $repertoire_dest créé."
fi

# Vérifier s'il y a des fichiers .gz dans le répertoire
shopt -s nullglob  # Permet d'éviter des erreurs si aucun fichier .gz n'existe
fichiers=("$repertoire"/*.gz)

if [ ${#fichiers[@]} -eq 0 ]; then
    echo "Aucun fichier .gz trouvé dans $repertoire"
    exit 1
fi

# boucle sur les fichiers .gz
for f in "${fichiers[@]}"; do
    # enlever l'extension .gz et extraire le fichier dans le répertoire de destination
    gunzip -c "$f" > "$repertoire_dest/$(basename "${f%.gz}")"
    
    echo "Fichier extrait et copié : $(basename "${f%.gz}")"
done

echo "Tous les fichiers ont été extraits et copiés dans le répertoire $repertoire_dest"

Répertoire 7_Article/data/GSE_extracted créé.


Fichier extrait et copié : GSM4594094_M11_barcodes.csv
Fichier extrait et copié : GSM4594094_M11_dcm.csv
Fichier extrait et copié : GSM4594094_M11_genes.csv
Fichier extrait et copié : GSM4594095_mRNA_M14_200.csv
Fichier extrait et copié : GSM4594095_mRNA_M14_ODannotation.csv
Fichier extrait et copié : GSM4594095_mRNA_M14_gene_names.csv
Fichier extrait et copié : GSM4594096_mRNA_M15_200.csv
Fichier extrait et copié : GSM4594096_mRNA_M15_ODannotation.csv
Fichier extrait et copié : GSM4594096_mRNA_M15_gene_names.csv
Tous les fichiers ont été extraits et copiés dans le répertoire 7_Article/data/GSE_extracted
